In [ ]:
import pandas as pd
import json
from pathlib import Path

def extract_json_data(file_path):
    """Read JSON (array or ndjson) and flatten nested 'readings' arrays.
    Returns DataFrame with one row per reading and columns:
    sensor_id, truck_id, timestamp, temperature (C), humidity, ...other fields
    """
    p = Path(file_path)
    if not p.exists():
        print("File tidak ditemukan:", file_path)
        return pd.DataFrame()

    # load JSON: support array or ndjson (one JSON object per line)
    text = p.read_text(encoding='utf-8').strip()
    if not text:
        return pd.DataFrame()

    records = []
    try:
        if text.startswith('['):
            data = json.loads(text)
            records = data if isinstance(data, list) else [data]
        else:
            # ndjson or single object
            lines = [l.strip() for l in text.splitlines() if l.strip()]
            if len(lines) == 1:
                obj = json.loads(lines[0])
                records = obj if isinstance(obj, list) else [obj]
            else:
                for line in lines:
                    try:
                        records.append(json.loads(line))
                    except json.JSONDecodeError:
                        continue
    except json.JSONDecodeError:
        print("JSON decode error")
        return pd.DataFrame()

    rows = []
    for item in records:
        if not isinstance(item, dict):
            continue
        base = {}
        # extract top-level identifiers if present
        base['sensor_id'] = item.get('sensor_id') or item.get('id') or None
        base['truck_id'] = item.get('truck_id') or item.get('truck') or None

        readings = item.get('readings')
        # if readings is a list -> one row per element
        if isinstance(readings, list):
            for r in readings:
                if not isinstance(r, dict):
                    rows.append({**base, 'reading': r})
                    continue
                row = dict(base)
                # timestamp preference: reading timestamp overrides item-level timestamp
                ts = r.get('timestamp') or r.get('time') or item.get('timestamp') or item.get('time')
                row['timestamp'] = ts
                # normalize possible temperature keys
                if 'temp' in r:
                    row['temperature'] = r.get('temp')
                elif 'temperature' in r:
                    row['temperature'] = r.get('temperature')
                if 'humidity' in r:
                    row['humidity'] = r.get('humidity')
                row.update({k:v for k,v in r.items() if k not in ('timestamp','time','temp','temperature','humidity')})
                rows.append(row)
        # if readings is dict -> merge
        elif isinstance(readings, dict):
            row = dict(base)
            ts = readings.get('timestamp') or readings.get('time') or item.get('timestamp') or item.get('time')
            row['timestamp'] = ts
            if 'temp' in readings:
                row['temperature'] = readings.get('temp')
            elif 'temperature' in readings:
                row['temperature'] = readings.get('temperature')
            if 'humidity' in readings:
                row['humidity'] = readings.get('humidity')
            row.update({k:v for k,v in readings.items() if k not in ('timestamp','time','temp','temperature','humidity')})
            rows.append(row)
        else:
            # no readings field -> treat item as single reading
            row = dict(base)
            ts = item.get('timestamp') or item.get('time')
            row['timestamp'] = ts
            if 'temp' in item:
                row['temperature'] = item.get('temp')
            elif 'temperature' in item:
                row['temperature'] = item.get('temperature')
            if 'humidity' in item:
                row['humidity'] = item.get('humidity')
            # include other top-level fields
            row.update({k:v for k,v in item.items() if k not in ('sensor_id','truck_id','readings','timestamp','time','temp','temperature','humidity')})
            rows.append(row)

    df = pd.DataFrame(rows)
    # normalize timestamp column
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    print(f"Extracted {len(df)} rows from {file_path}")
    return df

def transform_data(df):
    """Transform DataFrame:
    - interpret 'temperature' as Celsius
    - add temperature_f (F) and comfort_index = temp_c / humidity * 100 when possible
    """
    if df is None or df.empty:
        print("No data to transform")
        return pd.DataFrame()

    df = df.copy()

    # ensure numeric temperature and humidity
    if 'temperature' in df.columns:
        df['temperature_c'] = pd.to_numeric(df['temperature'], errors='coerce')
    else:
        df['temperature_c'] = pd.NA

    if 'humidity' in df.columns:
        df['humidity'] = pd.to_numeric(df['humidity'], errors='coerce')
    else:
        df['humidity'] = pd.NA

    # convert to Fahrenheit
    df['temperature_f'] = df['temperature_c'].apply(lambda c: round((c * 9/5) + 32, 2) if pd.notna(c) else pd.NA)

    # comfort index: temp_c / humidity * 100 (guard divide by zero)
    def compute_comfort(row):
        t = row.get('temperature_c')
        h = row.get('humidity')
        try:
            if pd.isna(t) or pd.isna(h) or h == 0:
                return pd.NA
            return round((t / h) * 100, 2)
        except Exception:
            return pd.NA

    df['comfort_index'] = df.apply(compute_comfort, axis=1)

    print(f"Transformed: {df.shape[0]} rows, {df.shape[1]} columns")
    return df

def load_to_csv(df, output_path):
    """Save DataFrame to CSV"""
    if df is None or df.empty:
        print("No data to save")
        return
    try:
        df.to_csv(output_path, index=False)
        print(f"Saved CSV: {output_path} ({df.shape[0]} rows)")
    except Exception as e:
        print("Error saving CSV:", e)

# Execute ETL pipeline
raw_data = extract_json_data('sensors.json')
transformed_data = transform_data(raw_data)
load_to_csv(transformed_data, 'etl_output.csv')

In [ ]:
# Execute ETL pipeline
raw_data = extract_json_data('uts-pid-2025/PID/datasets/sensors_2024-10-13.json')
transformed_data = transform_data(raw_data)
load_to_csv(transformed_data, 'etl_output.csv')